# KNN EVAL

In [1]:
# Set environment variables
import os

os.environ["DSPY_LOG_LEVEL"] = "DEBUG"
os.environ[
    "TOGETHER_API_KEY"] = SET THIS


In [2]:
import sys
import os
# Add the parent directory of 'brain' to the Python path
sys.path.append(os.path.abspath(".."))
from dspy import settings
from lms.together import Together

lm = Together(
    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
    temperature=0.5,
    max_tokens=1000,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1.2,
    stop=[
        "<|eot_id|>", "<|eom_id|>", "\n\n---\n\n", "\n\n---", "---", "\n---"
    ],
    # stop=["\n", "\n\n"],
)

settings.configure(lm=lm)

### Load training data

In [3]:
from brain.models import LabeledChatHistory
from brain.modules.chatter import ChatterModule

training_examples = LabeledChatHistory.load_labeled_histories(
    "../training_data/conversations.json")

### Evaluate the KNN Optimizer with the ChatterModule pipeline.

Note the error: 
```
2024/11/14 15:45:24 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'chat_history': ChatHistory(messages=[ChatMessage(from_creator=False, content="Hey! I saw you mentioned 'living slow' in your last video. Do you think it’s possible to do that while collecting typewriters and thinking about existential philosophy?", image_base64=None, timestamp=None), ChatMessage(from_creator=True, content="Absolutely! 'Living slow' is about finding purpose in what you do, even if it's pondering life’s mysteries or restoring a typewriter. It's more about presence than speed.", image_base64=None, timestamp=None), ChatMessage(from_creator=False, content='I love that! Maybe my typewriters are little time capsules of presence. Thanks for making me feel less weird about it.', image_base64=None, timestamp=None)]), 'response': 'Not weird at all! Embrace the things that make you feel deeply connected to the moment.'}) (input_keys={'chat_history'}) with None due to ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'general': 'Input should be a valid dictionary or instance of ChatHistory: chat_history (error type: model_type)'}).
```

However, the pipeline does work in chat_interface.py.... hmmmm Could it be a Together issue?

In [4]:
from brain.modules.knn_optimizer import KNNOptimizer

chatter = ChatterModule(examples=training_examples, use_filter=False)
knn_optimizer = chatter.optimizer
train_exemplar: LabeledChatHistory = knn_optimizer.labeled_examples[0]
print("Chat History: ", train_exemplar.chat_history)
print("Expected response: ", train_exemplar.response)

pred_response = chatter(chat_history=train_exemplar.chat_history).response
print("Actual response: ", pred_response.response)

Please use standard predictors, e.g. dspy.Predict and dspy.ChainOfThought.
They now support type annotations and other features of TypedPredictors and tend to work much better out of the box.
Please let us know if you face any issues: https://github.com/stanfordnlp/dspy/issues


Chat History:  THE FAN: Hey! I saw you mentioned 'living slow' in your last video. Do you think it’s possible to do that while collecting typewriters and thinking about existential philosophy?
YOU: Absolutely! 'Living slow' is about finding purpose in what you do, even if it's pondering life’s mysteries or restoring a typewriter. It's more about presence than speed.
THE FAN: I love that! Maybe my typewriters are little time capsules of presence. Thanks for making me feel less weird about it.
Expected response:  Not weird at all! Embrace the things that make you feel deeply connected to the moment.


  0%|          | 0/3 [00:00<?, ?it/s]WARNING:root:	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated, underperform, and are about to be deleted. ***
 		You are using the client Together, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
2024/11/14 16:04:21 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'chat_history': ChatHistory(messages=[ChatMessage(from_creator=False, content="Hey! I saw you mentioned 'living slow' in your last video. Do you think it’s possible to do that while collecting typewriters and thinking about existential philosophy?", image_base64=None, timestamp=None), ChatMessage(from_creator=True, content="Absolutely! 'Living slow' is about finding pur

Bootstrapped 0 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.


ValueError: ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'general': 'Input should be a valid dictionary or instance of ChatHistory: chat_history (error type: model_type)'})

## Evaluate

In [ ]:
from dspy.evaluate.evaluate import Evaluate
from dspy.evaluate import answer_passage_match

evaluate_knn_on_valset = Evaluate(devset=knn_optimizer.labeled_examples,
                                  num_threads=len(
                                      knn_optimizer.labeled_examples),
                                  display_progress=True,
                                  display_table=True,
                                  provide_traceback=True)
evaluate_knn_on_valset(chatter, answer_passage_match)